# Evolver Loop 2 - LB Feedback Analysis

## Key Findings
- CV-LB gap is EXACTLY 0 (70.6589 CV = 70.6589 LB)
- This means our local scoring is perfectly accurate
- Current best: 70.658891
- Target: 68.919154
- Gap: 1.74 points (2.53%)

## Critical Insight
Found a BETTER pre-optimized submission in snapshot 21165874980 with score 70.647306!

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import os
import json

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
BASE_TREE = Polygon(zip(TX, TY))

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree(x, y, deg):
    tree = affinity.rotate(BASE_TREE, deg, origin=(0, 0))
    tree = affinity.translate(tree, x, y)
    return tree

def get_score_for_n(df, n):
    prefix = f"{n:03d}_"
    n_rows = df[df['id'].str.startswith(prefix)]
    
    all_coords = []
    for _, row in n_rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        tree = create_tree(x, y, deg)
        coords = np.array(tree.exterior.coords)
        all_coords.append(coords)
    
    all_coords = np.vstack(all_coords)
    x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
    y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
    side = max(x_range, y_range)
    score = side**2 / n
    return side, score

print("Functions defined")

Functions defined


In [2]:
# Load the better submission from snapshot 21165874980
better_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21165874980/code/submission_candidates/candidate_001.csv')
current_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21164519357/code/exploration/santa-2025.csv')

print(f"Better submission rows: {len(better_df)}")
print(f"Current submission rows: {len(current_df)}")

Better submission rows: 20100
Current submission rows: 20100


In [3]:
# Compare scores for each N
print("Comparing scores for each N:")
print("N | Current | Better | Diff")
print("-" * 40)

total_current = 0
total_better = 0
improvements = []

for n in range(1, 201):
    _, current_score = get_score_for_n(current_df, n)
    _, better_score = get_score_for_n(better_df, n)
    total_current += current_score
    total_better += better_score
    diff = current_score - better_score
    if abs(diff) > 0.0001:
        improvements.append((n, current_score, better_score, diff))

print(f"\nTotal current: {total_current:.6f}")
print(f"Total better: {total_better:.6f}")
print(f"Improvement: {total_current - total_better:.6f}")

print(f"\nN values with improvements ({len(improvements)}):")
for n, curr, better, diff in sorted(improvements, key=lambda x: -x[3])[:20]:
    print(f"N={n}: {curr:.6f} -> {better:.6f} (diff: {diff:.6f})")

Comparing scores for each N:
N | Current | Better | Diff
----------------------------------------



Total current: 70.658891
Total better: 70.647306
Improvement: 0.011585

N values with improvements (20):
N=128: 0.343716 -> 0.340974 (diff: 0.002742)
N=94: 0.352271 -> 0.350076 (diff: 0.002195)
N=157: 0.341360 -> 0.340204 (diff: 0.001156)
N=14: 0.370454 -> 0.369543 (diff: 0.000911)
N=11: 0.375736 -> 0.374924 (diff: 0.000812)
N=193: 0.333760 -> 0.333341 (diff: 0.000418)
N=137: 0.343177 -> 0.342810 (diff: 0.000366)
N=123: 0.347904 -> 0.347565 (diff: 0.000340)
N=59: 0.360855 -> 0.360583 (diff: 0.000272)
N=115: 0.343570 -> 0.343333 (diff: 0.000237)
N=101: 0.349035 -> 0.348801 (diff: 0.000234)
N=192: 0.335301 -> 0.335077 (diff: 0.000223)
N=89: 0.346899 -> 0.346717 (diff: 0.000182)
N=32: 0.365592 -> 0.365417 (diff: 0.000175)
N=76: 0.351372 -> 0.351218 (diff: 0.000154)
N=74: 0.352850 -> 0.352709 (diff: 0.000141)
N=197: 0.335990 -> 0.335880 (diff: 0.000110)
N=57: 0.353634 -> 0.353528 (diff: 0.000106)
N=143: 0.341147 -> 0.341045 (diff: 0.000103)
N=172: 0.340526 -> 0.340425 (diff: 0.000101)


In [4]:
# Copy the better submission
import shutil
os.makedirs('/home/code/experiments/007_better_ensemble', exist_ok=True)
shutil.copy('/home/nonroot/snapshots/santa-2025/21165874980/code/submission_candidates/candidate_001.csv',
            '/home/code/experiments/007_better_ensemble/submission.csv')
shutil.copy('/home/code/experiments/007_better_ensemble/submission.csv',
            '/home/submission/submission.csv')
print("Copied better submission to /home/submission/submission.csv")

Copied better submission to /home/submission/submission.csv


In [ ]:
# Save metrics
metrics = {'cv_score': total_better}
with open('/home/code/experiments/007_better_ensemble/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Saved metrics: {metrics}")